## Assignment 4
### Basic Preparation
#### Importing packages and dataset

In [97]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from scipy import stats
import seaborn as sns
import statsmodels.api as sm
import sklearn
from sklearn import linear_model
from statsmodels.formula.api import ols
from sklearn.neighbors import KNeighborsClassifier 
from collections import defaultdict
from sklearn import svm # SVM classifier
from sklearn.neural_network import MLPClassifier # MLP classifier
from sklearn.model_selection import train_test_split,cross_val_score,cross_val_predict,KFold,GridSearchCV,RandomizedSearchCV
from sklearn import metrics
from sklearn.preprocessing import StandardScaler,OneHotEncoder,LabelEncoder
from sklearn.neighbors import KNeighborsClassifier 
from sklearn.tree import DecisionTreeClassifier 
from sklearn.ensemble import RandomForestClassifier 
from sklearn.naive_bayes import MultinomialNB, GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_curve # ROC Curves
from sklearn.metrics import auc # Calculating AUC for ROC's!
from sklearn.neural_network import MLPClassifier
from mlxtend.classifier import StackingClassifier
from sklearn import model_selection
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import SGD
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense,Dropout
import warnings
warnings.filterwarnings("ignore")

In [2]:
# Import original data

ss= pd.read_csv("skyserver.csv")
ss.head()

,objid,ra,dec,u,g,r,i,z,run,rerun,camcol,field,specobjid,class,redshift,plate,mjd,fiberid
0,1.237650e+18,183.531326,0.089693,19.47406,17.04240,15.94699,15.50342,15.22531,752,301,4,267,3.722360e+18,STAR,-0.000009,3306,54922,491
1,1.237650e+18,183.598371,0.135285,18.66280,17.21449,16.67637,16.48922,16.39150,752,301,4,267,3.638140e+17,STAR,-0.000055,323,51615,541
2,1.237650e+18,183.680207,0.126185,19.38298,18.19169,17.47428,17.08732,16.80125,752,301,4,268,3.232740e+17,GALAXY,0.123111,287,52023,513
3,1.237650e+18,183.870529,0.049911,17.76536,16.60272,16.16116,15.98233,15.90438,752,301,4,269,3.722370e+18,STAR,-0.000111,3306,54922,510
4,1.237650e+18,183.883288,0.102557,17.55025,16.26342,16.43869,16.55492,16.61326,752,301,4,269,3.722370e+18,STAR,0.000590,3306,54922,512


In [3]:
ss.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 18 columns):
objid        10000 non-null float64
ra           10000 non-null float64
dec          10000 non-null float64
u            10000 non-null float64
g            10000 non-null float64
r            10000 non-null float64
i            10000 non-null float64
z            10000 non-null float64
run          10000 non-null int64
rerun        10000 non-null int64
camcol       10000 non-null int64
field        10000 non-null int64
specobjid    10000 non-null float64
class        10000 non-null object
redshift     10000 non-null float64
plate        10000 non-null int64
mjd          10000 non-null int64
fiberid      10000 non-null int64
dtypes: float64(10), int64(7), object(1)
memory usage: 1.4+ MB


What I want to research on is 'class' column, which is the dependent value. So here I moved it to the far left of the table.

In [4]:
cl=ss['class']
ss.drop(labels=['class'], axis=1,inplace = True)
ss.insert(0, 'class', cl)
ss.head()

,class,objid,ra,dec,u,g,r,i,z,run,rerun,camcol,field,specobjid,redshift,plate,mjd,fiberid
0,STAR,1.237650e+18,183.531326,0.089693,19.47406,17.04240,15.94699,15.50342,15.22531,752,301,4,267,3.722360e+18,-0.000009,3306,54922,491
1,STAR,1.237650e+18,183.598371,0.135285,18.66280,17.21449,16.67637,16.48922,16.39150,752,301,4,267,3.638140e+17,-0.000055,323,51615,541
2,GALAXY,1.237650e+18,183.680207,0.126185,19.38298,18.19169,17.47428,17.08732,16.80125,752,301,4,268,3.232740e+17,0.123111,287,52023,513
3,STAR,1.237650e+18,183.870529,0.049911,17.76536,16.60272,16.16116,15.98233,15.90438,752,301,4,269,3.722370e+18,-0.000111,3306,54922,510
4,STAR,1.237650e+18,183.883288,0.102557,17.55025,16.26342,16.43869,16.55492,16.61326,752,301,4,269,3.722370e+18,0.000590,3306,54922,512


In [5]:
ss.describe()

,objid,ra,dec,u,g,r,i,z,run,rerun,camcol,field,specobjid,redshift,plate,mjd,fiberid
count,1.000000e+04,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.0,10000.000000,10000.000000,1.000000e+04,10000.000000,10000.000000,10000.000000,10000.000000
mean,1.237650e+18,175.529987,14.836148,18.619355,17.371931,16.840963,16.583579,16.422833,981.034800,301.0,3.648700,302.380100,1.645022e+18,0.143726,1460.986400,52943.533300,353.069400
std,1.577039e+05,47.783439,25.212207,0.828656,0.945457,1.067764,1.141805,1.203188,273.305024,0.0,1.666183,162.577763,2.013998e+18,0.388774,1788.778371,1511.150651,206.298149
min,1.237650e+18,8.235100,-5.382632,12.988970,12.799550,12.431600,11.947210,11.610410,308.000000,301.0,1.000000,11.000000,2.995780e+17,-0.004136,266.000000,51578.000000,1.000000
25%,1.237650e+18,157.370946,-0.539035,18.178035,16.815100,16.173333,15.853705,15.618285,752.000000,301.0,2.000000,184.000000,3.389248e+17,0.000081,301.000000,51900.000000,186.750000
50%,1.237650e+18,180.394514,0.404166,18.853095,17.495135,16.858770,16.554985,16.389945,756.000000,301.0,4.000000,299.000000,4.966580e+17,0.042591,441.000000,51997.000000,351.000000
75%,1.237650e+18,201.547279,35.649397,19.259232,18.010145,17.512675,17.258550,17.141447,1331.000000,301.0,5.000000,414.000000,2.881300e+18,0.092579,2559.000000,54468.000000,510.000000
max,1.237650e+18,260.884382,68.542265,19.599900,19.918970,24.802040,28.179630,22.833060,1412.000000,301.0,6.000000,768.000000,9.468830e+18,5.353854,8410.000000,57481.000000,1000.000000


#### Cleaning data

##### Brief Introduction about each column

objid = Object Identifier

ra = J2000 Right Ascension (r-band)-
    
    (The coordinates on the sky that correspond to longitude on Earth. Ra measures east and west on the celestial sphere and is like longitude on the Earth.)

dec = J2000 Declination (r-band)

    (The coordinates on the sky that correspond to latitude on Earth. Dec measures north and south on the celestial sphere and is like latitude on the Earth.)
    
u (ultraviolet)= better of DeV/Exp magnitude fit

g (green) = better of DeV/Exp magnitude fit

r (red) = better of DeV/Exp magnitude fit

i (Near infrared) = better of DeV/Exp magnitude fit

z (Infrared) = better of DeV/Exp magnitude fit

run = Run Number

rereun = Rerun Number

camcol = Camera column

field = Field number

specobjid = Object Identifier

class = object class (galaxy, star or quasar object)

redshift = Final Redshift

plate = plate number

mjd = MJD of observation

fiberid = fiber ID

From the definition of each columns, it's obvious that objid, specobjid and fiberid are useless to the prediction to  the class. So I chose to drop the three columns.

In [6]:
ss.drop(['objid','specobjid','fiberid','run','rerun','camcol','field'],axis=1,inplace=True)

In [7]:
ss.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 11 columns):
class       10000 non-null object
ra          10000 non-null float64
dec         10000 non-null float64
u           10000 non-null float64
g           10000 non-null float64
r           10000 non-null float64
i           10000 non-null float64
z           10000 non-null float64
redshift    10000 non-null float64
plate       10000 non-null int64
mjd         10000 non-null int64
dtypes: float64(8), int64(2), object(1)
memory usage: 859.5+ KB


In [8]:
ss.head()

,class,ra,dec,u,g,r,i,z,redshift,plate,mjd
0,STAR,183.531326,0.089693,19.47406,17.04240,15.94699,15.50342,15.22531,-0.000009,3306,54922
1,STAR,183.598371,0.135285,18.66280,17.21449,16.67637,16.48922,16.39150,-0.000055,323,51615
2,GALAXY,183.680207,0.126185,19.38298,18.19169,17.47428,17.08732,16.80125,0.123111,287,52023
3,STAR,183.870529,0.049911,17.76536,16.60272,16.16116,15.98233,15.90438,-0.000111,3306,54922
4,STAR,183.883288,0.102557,17.55025,16.26342,16.43869,16.55492,16.61326,0.000590,3306,54922


##### Label Encoding

On the ground that the datatype of 'class',the dependent value is object. In order to do later research, so I used label encoding here for the 'class' column.

In [9]:
le=LabelEncoder()

In [10]:
X=ss.iloc[:,1:]
y=ss.iloc[:,0]

In [11]:
y=le.fit_transform(y)
y=pd.DataFrame(y)

In [12]:
ss=pd.concat([y,X],axis=1,sort=False)
ss.rename(index=str, columns={0: "class"},inplace=True)
ss.head()

,class,ra,dec,u,g,r,i,z,redshift,plate,mjd
0,2,183.531326,0.089693,19.47406,17.04240,15.94699,15.50342,15.22531,-0.000009,3306,54922
1,2,183.598371,0.135285,18.66280,17.21449,16.67637,16.48922,16.39150,-0.000055,323,51615
2,0,183.680207,0.126185,19.38298,18.19169,17.47428,17.08732,16.80125,0.123111,287,52023
3,2,183.870529,0.049911,17.76536,16.60272,16.16116,15.98233,15.90438,-0.000111,3306,54922
4,2,183.883288,0.102557,17.55025,16.26342,16.43869,16.55492,16.61326,0.000590,3306,54922


##### detecting empty values

In [13]:
ss.isnull().sum()

class       0
ra          0
dec         0
u           0
g           0
r           0
i           0
z           0
redshift    0
plate       0
mjd         0
dtype: int64

From the result, there is no empty value.

##### Preprocessing: Scaling

In [14]:
def scaled_df(df):
    scaled = pd.DataFrame()
    for item in df:
        if item in df.select_dtypes(include=[np.float]):
            scaled[item] = ((df[item] - df[item].min()) / 
            (df[item].max() - df[item].min()))
        else: 
            scaled[item] = df[item]
    return scaled
ss_scaled= scaled_df(ss)

In [15]:
ss_scaled.head()

,class,ra,dec,u,g,r,i,z,redshift,plate,mjd
0,2,0.693832,0.074025,0.980965,0.595954,0.284177,0.219081,0.322108,0.000770,3306,54922
1,2,0.694098,0.074642,0.858250,0.620126,0.343138,0.279811,0.426021,0.000762,323,51615
2,0,0.694422,0.074519,0.967188,0.757385,0.407640,0.316657,0.462532,0.023749,287,52023
3,2,0.695175,0.073487,0.722499,0.534197,0.301490,0.248584,0.382616,0.000751,3306,54922
4,2,0.695225,0.074199,0.689960,0.486538,0.323925,0.283858,0.445782,0.000882,3306,54922


In [16]:
ss_scaled.describe()

,class,ra,dec,u,g,r,i,z,redshift,plate,mjd
count,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000
mean,0.915400,0.662163,0.273504,0.851678,0.642241,0.356444,0.285624,0.428813,0.027597,1460.986400,52943.533300
std,0.952856,0.189130,0.341052,0.125346,0.132800,0.086316,0.070341,0.107211,0.072560,1788.778371,1511.150651
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,266.000000,51578.000000
25%,0.000000,0.590288,0.065521,0.784922,0.564028,0.302474,0.240660,0.357124,0.000787,301.000000,51900.000000
50%,1.000000,0.681417,0.078279,0.887035,0.659546,0.357883,0.283862,0.425883,0.008721,441.000000,51997.000000
75%,2.000000,0.765140,0.555050,0.948469,0.731885,0.410743,0.327206,0.492846,0.018051,2559.000000,54468.000000
max,2.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,8410.000000,57481.000000


##### Using zscore to detect outlier and remove them

In [17]:
z=np.abs(stats.zscore(ss_scaled))
z

array([[1.13831973, 0.16745842, 0.58492272, ..., 0.36973112, 1.03148936,
        1.30931064],
       [1.13831973, 0.16886159, 0.58311429, ..., 0.36984929, 0.63621258,
        0.87919741],
       [0.96073933, 0.17057433, 0.58347525, ..., 0.05302706, 0.65633905,
        0.60919097],
       ...,
       [1.13831973, 0.92039468, 1.46090664, ..., 0.37074104, 3.26608697,
        2.69309363],
       [0.96073933, 0.92197294, 1.46432109, ..., 0.33364691, 0.56688806,
        0.70581092],
       [0.96073933, 0.91804124, 1.46639319, ..., 0.06510125, 0.56688806,
        0.70581092]])

In [18]:
print(np.where(z>3))

(array([  45,  109,  129, ..., 9992, 9994, 9997]), array([8, 8, 8, ..., 9, 9, 9]))


In [19]:
ss_scaled=ss_scaled[(z<3).all(axis=1)]

In [20]:
ss_scaled.info()

<class 'pandas.core.frame.DataFrame'>
Index: 8713 entries, 0 to 9999
Data columns (total 11 columns):
class       8713 non-null int64
ra          8713 non-null float64
dec         8713 non-null float64
u           8713 non-null float64
g           8713 non-null float64
r           8713 non-null float64
i           8713 non-null float64
z           8713 non-null float64
redshift    8713 non-null float64
plate       8713 non-null int64
mjd         8713 non-null int64
dtypes: float64(8), int64(3)
memory usage: 816.8+ KB


### Supervised Learning


In [23]:
X=ss_scaled.iloc[:,1:]
y=ss_scaled[['class']]

In [24]:
X_train,X_test,y_train,y_test=train_test_split(X,y,random_state=0)

In [25]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(6534, 10)
(2179, 10)
(6534, 1)
(2179, 1)


#### Decision Tree

In [26]:
dtm=DecisionTreeClassifier()

In [27]:
tunedParamDtm={'random_state':[10,20,30,40,50,60,70,80,90,100],'criterion':['gini','entropy'],'max_features':[2,3,4,5,6,7,8,9,10]}

In [28]:
dtmTuned=GridSearchCV(dtm,tunedParamDtm,cv=10)

In [29]:
dtmTuned.fit(X_train,y_train)

GridSearchCV(cv=10, error_score='raise',
       estimator=DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'random_state': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100], 'criterion': ['gini', 'entropy'], 'max_features': [2, 3, 4, 5, 6, 7, 8, 9, 10]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [30]:
print(dtmTuned.best_params_)

{'criterion': 'entropy', 'max_features': 8, 'random_state': 20}


In [31]:
dtmTuned.best_score_

0.9882154882154882

In [32]:
dtmN=DecisionTreeClassifier(criterion='entropy',max_features=8,random_state=20)

print("KfoldCrossVal mean score using decision tree is %s" %cross_val_score(dtmN,X,y,cv=10).mean())


sm= dtmN.fit(X_train, y_train)

y_pred = sm.predict(X_test)
print("Accuracy score using Decision Tree is %s" %metrics.accuracy_score(y_test, y_pred))

KfoldCrossVal mean score using decision tree is 0.9816404096967878
Accuracy score using Decision Tree is 0.9857732905002294


* Which hyper-parameters are important?

For Decision tree, there are some important hyper-parameters: random_state, criterion, max_features.

criterion : The function to measure the quality of a split. Supported criteria are “gini” for the Gini impurity and “entropy” for the information gain. Note: this parameter is tree-specific.

max_features : The number of features to consider when looking for the best split.

random_state : If int, random_state is the seed used by the random number generator.


* What hyper-parameter values work best? 

Through grid search, the best values for decision tree of criterion is entropy and max_features is 8 and random_state is 20.


#### Random Forest

In [33]:
rdf=RandomForestClassifier()

In [34]:
tunedParamRdf={'random_state':[10,20,30,40,50],'criterion':['gini','entropy'],'n_estimators':[20,30,40,50,60,70],'max_features':[2,3,4,5,6,7,8,9,10]}

In [35]:
rdfTuned=RandomizedSearchCV(rdf,tunedParamRdf,cv=10)

In [36]:
rdfTuned.fit(X_train,y_train)

RandomizedSearchCV(cv=10, error_score='raise',
          estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False),
          fit_params=None, iid=True, n_iter=10, n_jobs=1,
          param_distributions={'random_state': [10, 20, 30, 40, 50], 'criterion': ['gini', 'entropy'], 'n_estimators': [20, 30, 40, 50, 60, 70], 'max_features': [2, 3, 4, 5, 6, 7, 8, 9, 10]},
          pre_dispatch='2*n_jobs', random_state=None, refit=True,
          return_train_score='warn', scoring=None, verbose=0)

In [37]:
print(rdfTuned.best_params_)

{'random_state': 50, 'n_estimators': 60, 'max_features': 8, 'criterion': 'entropy'}


In [38]:
rdfTuned.best_score_

0.9905111723293542

In [39]:
rdfN=RandomForestClassifier(criterion='entropy',max_features=7,n_estimators=30,random_state=40)

print("KfoldCrossVal mean score using random forest is %s" %cross_val_score(rdfN,X,y,cv=10).mean())


sm= rdfN.fit(X_train, y_train)

y_pred = sm.predict(X_test)
print("Accuracy score using Random Forest is %s" %metrics.accuracy_score(y_test, y_pred))

KfoldCrossVal mean score using random forest is 0.9892093628931932
Accuracy score using Random Forest is 0.9880679210647085


* Which hyper-parameters are important?

For random forest, there are some important hyper-parameters: random_state, criterion, max_features, n_estimators.

criterion : The function to measure the quality of a split. Supported criteria are “gini” for the Gini impurity and “entropy” for the information gain. Note: this parameter is tree-specific.

max_features : The number of features to consider when looking for the best split.

random_state : If int, random_state is the seed used by the random number generator.

n_estimators : The number of trees in the forest.

* What hyper-parameter values work best? 

Through grid search, the best values for random forest of criterion is entropy and max_features is 7 and random_state is 40, n_estimators is 30.

#### K-Nearest Neighbors

Cause there is only one important hyper-parameter, here I use a loop to find the best k for K-Nearest Neighbors algorithm.

In [40]:
knn_k=[]
for i in range(0,55):
    #The kNN classification problem is to find the k nearest data points in a data set to a given query data point. 
    #The point is then assigned to the group by a majority "vote." 
    #For this reason, pick an odd k is prefered as the odd vote can break ties. 
    if(i%2 != 0):
        knn_k.append(i)
        
cross_vals=[]
for k in knn_k:
    knn=KNeighborsClassifier(n_neighbors=k)
    scores=cross_val_score(knn,X_train,y_train,cv=10,scoring='accuracy')
    cross_vals.append(scores.mean())

optimal_k=knn_k[cross_vals.index(min(cross_vals))]
print("Optimal K is {0}".format(optimal_k))

Optimal K is 37


In [41]:
knn=KNeighborsClassifier(n_neighbors=37)

print("KfoldCrossVal mean score using K-Nearest Neighbors is %s" %cross_val_score(knn,X,y,cv=10).mean())


sm= knn.fit(X_train, y_train)

y_pred = sm.predict(X_test)
print("Accuracy score using K-Nearest Neighbors is %s" %metrics.accuracy_score(y_test, y_pred))

KfoldCrossVal mean score using K-Nearest Neighbors is 0.8325530738130414
Accuracy score using K-Nearest Neighbors is 0.833409821018816


* Which hyper-parameters are important?

For K-Nearest Neighbors algorithm, k matters.

K: Number of neighbors to use by default for kneighbors queries.

* What hyper-parameter values work best? 

Through grid search, the best values of k is 37.

#### Naive Bayes

Here using RandomizedSearchCV to find the best values of hyper-parameters for naive bayes model.

In [42]:
mnb=MultinomialNB()

In [43]:
TunedParamMnb={'alpha':[0.1,1,10,105,110,115],'fit_prior':[True,False]}

In [44]:
MnbTuned=GridSearchCV(mnb,TunedParamMnb,cv=10)

In [45]:
MnbTuned.fit(X_train,y_train)

GridSearchCV(cv=10, error_score='raise',
       estimator=MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'alpha': [0.1, 1, 10, 105, 110, 115], 'fit_prior': [True, False]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [46]:
MnbTuned.best_params_

{'alpha': 110, 'fit_prior': True}

In [47]:
MnbTuned.best_score_

0.7861952861952862

* Which hyper-parameters are important?

For MutinomialNB, alpha and fit_prior are important hyper-parameters.

alpha : Additive (Laplace/Lidstone) smoothing parameter (0 for no smoothing).

fit_prior :Whether to learn class prior probabilities or not. If false, a uniform prior will be used.

* What hyper-parameter values work best? 

Through GridSearchCV, the best values of alpha is 110, fit_prior True.

##### * Which supervised learner works best on the test data? 

From above, I used some algoritms of supervised learning, decision tree, random forest, K-Nearest Neighbors and Naive Bayes.

Through comparing the accuracy scores of each algorithm, random forest works best on the test data.

## Neural networks can be used for supervised learning

In [48]:
n_classes = 3
y_train=to_categorical(y_train, n_classes)
y_test=to_categorical(y_test, n_classes)

In [49]:
y_test[0:5]

array([[1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       [1., 0., 0.],
       [1., 0., 0.]], dtype=float32)

##### A shallow neutral network with 'sigmoid' activation and 'mean_squared_error' loss function.

In [50]:
def shallow_net_A(n=100,i=10,o=3):
    # create simple one dense layer net
    # default 100 neurons, input 10, output 3
    net = Sequential()
    net.add(Dense(n, activation='sigmoid', input_shape=(i,)))
    net.add(Dense(o, activation='softmax'))
    # Compile net
    net.compile(loss='mean_squared_error', optimizer=SGD(lr=0.01), metrics=['accuracy'])
    return net

In [51]:
nna=shallow_net_A()

In [52]:
nna.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 100)               1100      
_________________________________________________________________
dense_2 (Dense)              (None, 3)                 303       
Total params: 1,403
Trainable params: 1,403
Non-trainable params: 0
_________________________________________________________________


In [53]:
nna.fit(X_train, y_train, batch_size=128, epochs=99, verbose=1, validation_data=(X_test, y_test))

Train on 6534 samples, validate on 2179 samples
Epoch 1/99
6534/6534 [==============================] - 0s 53us/step - loss: 0.2216 - acc: 0.4480 - val_loss: 0.1899 - val_acc: 0.5571
Epoch 2/99
6534/6534 [==============================] - 0s 15us/step - loss: 0.1883 - acc: 0.5450 - val_loss: 0.1850 - val_acc: 0.5571
Epoch 3/99
6534/6534 [==============================] - 0s 13us/step - loss: 0.1851 - acc: 0.5450 - val_loss: 0.1836 - val_acc: 0.5571
Epoch 4/99
6534/6534 [==============================] - 0s 11us/step - loss: 0.1835 - acc: 0.5450 - val_loss: 0.1818 - val_acc: 0.5571
Epoch 5/99
6534/6534 [==============================] - 0s 10us/step - loss: 0.1821 - acc: 0.5450 - val_loss: 0.1807 - val_acc: 0.5571
Epoch 6/99
6534/6534 [==============================] - 0s 10us/step - loss: 0.1809 - acc: 0.5450 - val_loss: 0.1793 - val_acc: 0.5571
Epoch 7/99
6534/6534 [==============================] - 0s 9us/step - loss: 0.1797 - acc: 0.5450 - val_loss: 0.1783 - val_acc: 0.5571
Epoch 8/

6534/6534 [==============================] - 0s 8us/step - loss: 0.1469 - acc: 0.7227 - val_loss: 0.1459 - val_acc: 0.7292
Epoch 62/99
6534/6534 [==============================] - 0s 8us/step - loss: 0.1467 - acc: 0.7227 - val_loss: 0.1461 - val_acc: 0.7292
Epoch 63/99
6534/6534 [==============================] - 0s 9us/step - loss: 0.1464 - acc: 0.7227 - val_loss: 0.1457 - val_acc: 0.7292
Epoch 64/99
6534/6534 [==============================] - 0s 11us/step - loss: 0.1461 - acc: 0.7227 - val_loss: 0.1458 - val_acc: 0.7292
Epoch 65/99
6534/6534 [==============================] - 0s 12us/step - loss: 0.1460 - acc: 0.7227 - val_loss: 0.1456 - val_acc: 0.7292
Epoch 66/99
6534/6534 [==============================] - 0s 11us/step - loss: 0.1457 - acc: 0.7227 - val_loss: 0.1446 - val_acc: 0.7292
Epoch 67/99
6534/6534 [==============================] - 0s 11us/step - loss: 0.1454 - acc: 0.7227 - val_loss: 0.1444 - val_acc: 0.7292
Epoch 68/99
6534/6534 [==============================] - 0s 11u

In [54]:
# 73% accuracy after 99 epochs
nna.evaluate(X_test, y_test)

2179/2179 [==============================] - 0s 19us/step


[0.1398930752725982, 0.729233593391464]

##### A shallow neutral network with 'relu' activation and 'mean_squared_error' loss function.

In [64]:
def shallow_net_B(n=100,i=10,o=3):
    # create simple one dense layer net
    # default 100 neurons, input 10, output 3
    net = Sequential()
    net.add(Dense(n, activation='relu', input_shape=(i,)))
    net.add(Dense(o, activation='softmax'))
    # Compile net
    net.compile(loss='mean_squared_error', optimizer=SGD(lr=0.01), metrics=['accuracy'])
    return net

In [65]:
nnb=shallow_net_B()

In [66]:
nnb.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_7 (Dense)              (None, 100)               1100      
_________________________________________________________________
dense_8 (Dense)              (None, 3)                 303       
Total params: 1,403
Trainable params: 1,403
Non-trainable params: 0
_________________________________________________________________


In [67]:
nnb.fit(X_train, y_train, batch_size=128, epochs=99, verbose=1, validation_data=(X_test, y_test))

Train on 6534 samples, validate on 2179 samples
Epoch 1/99
6534/6534 [==============================] - 0s 41us/step - loss: 0.3033 - acc: 0.5450 - val_loss: 0.2952 - val_acc: 0.5571
Epoch 2/99
6534/6534 [==============================] - 0s 10us/step - loss: 0.3033 - acc: 0.5450 - val_loss: 0.2952 - val_acc: 0.5571
Epoch 3/99
6534/6534 [==============================] - 0s 14us/step - loss: 0.3033 - acc: 0.5450 - val_loss: 0.2952 - val_acc: 0.5571
Epoch 4/99
6534/6534 [==============================] - 0s 17us/step - loss: 0.3033 - acc: 0.5450 - val_loss: 0.2952 - val_acc: 0.5571
Epoch 5/99
6534/6534 [==============================] - 0s 16us/step - loss: 0.3033 - acc: 0.5450 - val_loss: 0.2952 - val_acc: 0.5571
Epoch 6/99
6534/6534 [==============================] - 0s 13us/step - loss: 0.3033 - acc: 0.5450 - val_loss: 0.2952 - val_acc: 0.5571
Epoch 7/99
6534/6534 [==============================] - 0s 11us/step - loss: 0.3033 - acc: 0.5450 - val_loss: 0.2952 - val_acc: 0.5571
Epoch 8

Epoch 61/99
6534/6534 [==============================] - 0s 14us/step - loss: 0.3033 - acc: 0.5450 - val_loss: 0.2952 - val_acc: 0.5571
Epoch 62/99
6534/6534 [==============================] - 0s 15us/step - loss: 0.3033 - acc: 0.5450 - val_loss: 0.2952 - val_acc: 0.5571
Epoch 63/99
6534/6534 [==============================] - 0s 12us/step - loss: 0.3033 - acc: 0.5450 - val_loss: 0.2952 - val_acc: 0.5571
Epoch 64/99
6534/6534 [==============================] - 0s 16us/step - loss: 0.3033 - acc: 0.5450 - val_loss: 0.2952 - val_acc: 0.5571
Epoch 65/99
6534/6534 [==============================] - 0s 12us/step - loss: 0.3033 - acc: 0.5450 - val_loss: 0.2952 - val_acc: 0.5571
Epoch 66/99
6534/6534 [==============================] - 0s 13us/step - loss: 0.3033 - acc: 0.5450 - val_loss: 0.2952 - val_acc: 0.5571
Epoch 67/99
6534/6534 [==============================] - 0s 14us/step - loss: 0.3033 - acc: 0.5450 - val_loss: 0.2952 - val_acc: 0.5571
Epoch 68/99
6534/6534 [=========================

In [68]:
# 56% accuracy after 99 epochs
nnb.evaluate(X_test, y_test)

2179/2179 [==============================] - 0s 16us/step


[0.29524247727163233, 0.5571363010828841]

##### A shallow neutral network with 'tanh' activation and 'mean_squared_error' loss function.

In [69]:
def shallow_net_C(n=100,i=10,o=3):
    # create simple one dense layer net
    # default 100 neurons, input 10, output 3
    net = Sequential()
    net.add(Dense(n, activation='tanh', input_shape=(i,)))
    net.add(Dense(o, activation='softmax'))
    # Compile net
    net.compile(loss='mean_squared_error', optimizer=SGD(lr=0.01), metrics=['accuracy'])
    return net

In [70]:
nnc=shallow_net_C()

In [71]:
nnc.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_9 (Dense)              (None, 100)               1100      
_________________________________________________________________
dense_10 (Dense)             (None, 3)                 303       
Total params: 1,403
Trainable params: 1,403
Non-trainable params: 0
_________________________________________________________________


In [72]:
nnc.fit(X_train, y_train, batch_size=128, epochs=99, verbose=1, validation_data=(X_test, y_test))

Train on 6534 samples, validate on 2179 samples
Epoch 1/99
6534/6534 [==============================] - 0s 50us/step - loss: 0.2277 - acc: 0.5450 - val_loss: 0.2131 - val_acc: 0.5571
Epoch 2/99
6534/6534 [==============================] - 0s 13us/step - loss: 0.2105 - acc: 0.5450 - val_loss: 0.2036 - val_acc: 0.5571
Epoch 3/99
6534/6534 [==============================] - 0s 20us/step - loss: 0.2002 - acc: 0.5450 - val_loss: 0.1952 - val_acc: 0.5571
Epoch 4/99
6534/6534 [==============================] - 0s 15us/step - loss: 0.1899 - acc: 0.5447 - val_loss: 0.1835 - val_acc: 0.5571
Epoch 5/99
6534/6534 [==============================] - 0s 13us/step - loss: 0.1799 - acc: 0.5473 - val_loss: 0.1747 - val_acc: 0.5590
Epoch 6/99
6534/6534 [==============================] - 0s 15us/step - loss: 0.1703 - acc: 0.5478 - val_loss: 0.1651 - val_acc: 0.5590
Epoch 7/99
6534/6534 [==============================] - 0s 12us/step - loss: 0.1617 - acc: 0.5885 - val_loss: 0.1586 - val_acc: 0.5590
Epoch 8

Epoch 61/99
6534/6534 [==============================] - 0s 16us/step - loss: 0.0990 - acc: 0.8252 - val_loss: 0.0993 - val_acc: 0.8256
Epoch 62/99
6534/6534 [==============================] - 0s 17us/step - loss: 0.0989 - acc: 0.8252 - val_loss: 0.0997 - val_acc: 0.8256
Epoch 63/99
6534/6534 [==============================] - 0s 16us/step - loss: 0.0989 - acc: 0.8252 - val_loss: 0.0992 - val_acc: 0.8256
Epoch 64/99
6534/6534 [==============================] - 0s 14us/step - loss: 0.0988 - acc: 0.8252 - val_loss: 0.0991 - val_acc: 0.8256
Epoch 65/99
6534/6534 [==============================] - 0s 15us/step - loss: 0.0988 - acc: 0.8252 - val_loss: 0.0993 - val_acc: 0.8256
Epoch 66/99
6534/6534 [==============================] - 0s 14us/step - loss: 0.0987 - acc: 0.8252 - val_loss: 0.0990 - val_acc: 0.8256
Epoch 67/99
6534/6534 [==============================] - 0s 15us/step - loss: 0.0987 - acc: 0.8252 - val_loss: 0.0990 - val_acc: 0.8256
Epoch 68/99
6534/6534 [=========================

In [73]:
# 83% accuracy after 99 epochs
nnc.evaluate(X_test, y_test)

2179/2179 [==============================] - 0s 26us/step


[0.09803839303901869, 0.825608077099587]

##### A shallow neutral network with 'sigmoid' activation and 'categorical_crossentropy' loss function.

In [74]:
def shallow_net_D(n=100,i=10,o=3):
    # create simple one dense layer net
    # default 100 neurons, input 10, output 3
    net = Sequential()
    net.add(Dense(n, activation='sigmoid', input_shape=(i,)))
    net.add(Dense(o, activation='softmax'))
    # Compile net
    net.compile(loss='categorical_crossentropy', optimizer=SGD(lr=0.01), metrics=['accuracy'])
    return net

In [75]:
nnd=shallow_net_D()

In [76]:
nnd.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_11 (Dense)             (None, 100)               1100      
_________________________________________________________________
dense_12 (Dense)             (None, 3)                 303       
Total params: 1,403
Trainable params: 1,403
Non-trainable params: 0
_________________________________________________________________


In [77]:
nnd.fit(X_train, y_train, batch_size=128, epochs=99, verbose=1, validation_data=(X_test, y_test))

Train on 6534 samples, validate on 2179 samples
Epoch 1/99
6534/6534 [==============================] - 0s 66us/step - loss: 0.8607 - acc: 0.5450 - val_loss: 0.8418 - val_acc: 0.5571
Epoch 2/99
6534/6534 [==============================] - 0s 14us/step - loss: 0.8415 - acc: 0.5450 - val_loss: 0.8484 - val_acc: 0.5571
Epoch 3/99
6534/6534 [==============================] - 0s 18us/step - loss: 0.8416 - acc: 0.5450 - val_loss: 0.8391 - val_acc: 0.5571
Epoch 4/99
6534/6534 [==============================] - 0s 14us/step - loss: 0.8416 - acc: 0.5450 - val_loss: 0.8656 - val_acc: 0.3947
Epoch 5/99
6534/6534 [==============================] - 0s 17us/step - loss: 0.8418 - acc: 0.5401 - val_loss: 0.8490 - val_acc: 0.5571
Epoch 6/99
6534/6534 [==============================] - 0s 15us/step - loss: 0.8421 - acc: 0.5450 - val_loss: 0.8409 - val_acc: 0.5571
Epoch 7/99
6534/6534 [==============================] - 0s 16us/step - loss: 0.8417 - acc: 0.5450 - val_loss: 0.8475 - val_acc: 0.5571
Epoch 8

6534/6534 [==============================] - 0s 16us/step - loss: 0.8414 - acc: 0.5450 - val_loss: 0.8476 - val_acc: 0.5571
Epoch 61/99
6534/6534 [==============================] - 0s 15us/step - loss: 0.8424 - acc: 0.5450 - val_loss: 0.8452 - val_acc: 0.5571
Epoch 62/99
6534/6534 [==============================] - 0s 15us/step - loss: 0.8414 - acc: 0.5450 - val_loss: 0.8453 - val_acc: 0.5571
Epoch 63/99
6534/6534 [==============================] - 0s 14us/step - loss: 0.8414 - acc: 0.5450 - val_loss: 0.8392 - val_acc: 0.5571
Epoch 64/99
6534/6534 [==============================] - 0s 14us/step - loss: 0.8413 - acc: 0.5450 - val_loss: 0.8480 - val_acc: 0.5571
Epoch 65/99
6534/6534 [==============================] - 0s 15us/step - loss: 0.8415 - acc: 0.5450 - val_loss: 0.8491 - val_acc: 0.5571
Epoch 66/99
6534/6534 [==============================] - 0s 13us/step - loss: 0.8417 - acc: 0.5450 - val_loss: 0.8425 - val_acc: 0.5571
Epoch 67/99
6534/6534 [==============================] - 0s 

In [79]:
# 56% accuracy after 99 epochs
nnd.evaluate(X_test, y_test)

2179/2179 [==============================] - 0s 21us/step


[0.858998554871136, 0.5571363010828841]

##### A shallow neutral network with 'relu' activation and 'categorical_crossentropy' loss function.

In [80]:
def shallow_net_E(n=100,i=10,o=3):
    # create simple one dense layer net
    # default 100 neurons, input 10, output 3
    net = Sequential()
    net.add(Dense(n, activation='relu', input_shape=(i,)))
    net.add(Dense(o, activation='softmax'))
    # Compile net
    net.compile(loss='categorical_crossentropy', optimizer=SGD(lr=0.01), metrics=['accuracy'])
    return net

In [81]:
nne=shallow_net_E()

In [82]:
nne.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_13 (Dense)             (None, 100)               1100      
_________________________________________________________________
dense_14 (Dense)             (None, 3)                 303       
Total params: 1,403
Trainable params: 1,403
Non-trainable params: 0
_________________________________________________________________


In [83]:
nne.fit(X_train, y_train, batch_size=128, epochs=99, verbose=1, validation_data=(X_test, y_test))

Train on 6534 samples, validate on 2179 samples
Epoch 1/99
6534/6534 [==============================] - 0s 68us/step - loss: 7.3338 - acc: 0.5450 - val_loss: 7.1381 - val_acc: 0.5571
Epoch 2/99
6534/6534 [==============================] - 0s 15us/step - loss: 7.3338 - acc: 0.5450 - val_loss: 7.1381 - val_acc: 0.5571
Epoch 3/99
6534/6534 [==============================] - 0s 15us/step - loss: 7.3338 - acc: 0.5450 - val_loss: 7.1381 - val_acc: 0.5571
Epoch 4/99
6534/6534 [==============================] - 0s 17us/step - loss: 7.3338 - acc: 0.5450 - val_loss: 7.1381 - val_acc: 0.5571
Epoch 5/99
6534/6534 [==============================] - 0s 18us/step - loss: 7.3338 - acc: 0.5450 - val_loss: 7.1381 - val_acc: 0.5571
Epoch 6/99
6534/6534 [==============================] - 0s 16us/step - loss: 7.3338 - acc: 0.5450 - val_loss: 7.1381 - val_acc: 0.5571
Epoch 7/99
6534/6534 [==============================] - 0s 17us/step - loss: 7.3338 - acc: 0.5450 - val_loss: 7.1381 - val_acc: 0.5571
Epoch 8

Epoch 61/99
6534/6534 [==============================] - 0s 14us/step - loss: 7.3338 - acc: 0.5450 - val_loss: 7.1381 - val_acc: 0.5571
Epoch 62/99
6534/6534 [==============================] - 0s 17us/step - loss: 7.3338 - acc: 0.5450 - val_loss: 7.1381 - val_acc: 0.5571
Epoch 63/99
6534/6534 [==============================] - 0s 16us/step - loss: 7.3338 - acc: 0.5450 - val_loss: 7.1381 - val_acc: 0.5571
Epoch 64/99
6534/6534 [==============================] - 0s 15us/step - loss: 7.3338 - acc: 0.5450 - val_loss: 7.1381 - val_acc: 0.5571
Epoch 65/99
6534/6534 [==============================] - 0s 15us/step - loss: 7.3338 - acc: 0.5450 - val_loss: 7.1381 - val_acc: 0.5571
Epoch 66/99
6534/6534 [==============================] - 0s 15us/step - loss: 7.3338 - acc: 0.5450 - val_loss: 7.1381 - val_acc: 0.5571
Epoch 67/99
6534/6534 [==============================] - 0s 17us/step - loss: 7.3338 - acc: 0.5450 - val_loss: 7.1381 - val_acc: 0.5571
Epoch 68/99
6534/6534 [=========================

In [84]:
# 56% accuracy after 99 epochs
nne.evaluate(X_test, y_test)

2179/2179 [==============================] - 0s 17us/step


[7.138119484208603, 0.5571363010828841]

##### A shallow neutral network with 'tanh' activation and 'categorical_crossentropy' loss function.

In [85]:
def shallow_net_F(n=100,i=10,o=3):
    # create simple one dense layer net
    # default 100 neurons, input 10, output 3
    net = Sequential()
    net.add(Dense(n, activation='tanh', input_shape=(i,)))
    net.add(Dense(o, activation='softmax'))
    # Compile net
    net.compile(loss='categorical_crossentropy', optimizer=SGD(lr=0.01), metrics=['accuracy'])
    return net

In [86]:
nnf=shallow_net_F()

In [87]:
nnf.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_15 (Dense)             (None, 100)               1100      
_________________________________________________________________
dense_16 (Dense)             (None, 3)                 303       
Total params: 1,403
Trainable params: 1,403
Non-trainable params: 0
_________________________________________________________________


In [88]:
nnf.fit(X_train, y_train, batch_size=128, epochs=99, verbose=1, validation_data=(X_test, y_test))

Train on 6534 samples, validate on 2179 samples
Epoch 1/99
6534/6534 [==============================] - 0s 64us/step - loss: 0.8617 - acc: 0.5725 - val_loss: 0.8407 - val_acc: 0.5571
Epoch 2/99
6534/6534 [==============================] - 0s 15us/step - loss: 0.8425 - acc: 0.5450 - val_loss: 0.8464 - val_acc: 0.5571
Epoch 3/99
6534/6534 [==============================] - 0s 17us/step - loss: 0.8417 - acc: 0.5450 - val_loss: 0.8706 - val_acc: 0.5571
Epoch 4/99
6534/6534 [==============================] - 0s 17us/step - loss: 0.8419 - acc: 0.5450 - val_loss: 0.8550 - val_acc: 0.5571
Epoch 5/99
6534/6534 [==============================] - 0s 16us/step - loss: 0.8421 - acc: 0.5450 - val_loss: 0.8873 - val_acc: 0.5571
Epoch 6/99
6534/6534 [==============================] - 0s 16us/step - loss: 0.8434 - acc: 0.5450 - val_loss: 0.8420 - val_acc: 0.5571
Epoch 7/99
6534/6534 [==============================] - 0s 15us/step - loss: 0.8418 - acc: 0.5450 - val_loss: 0.9208 - val_acc: 0.3947
Epoch 8

6534/6534 [==============================] - 0s 16us/step - loss: 0.8410 - acc: 0.5419 - val_loss: 0.8529 - val_acc: 0.5576
Epoch 61/99
6534/6534 [==============================] - 0s 15us/step - loss: 0.8422 - acc: 0.5496 - val_loss: 0.8478 - val_acc: 0.5571
Epoch 62/99
6534/6534 [==============================] - 0s 16us/step - loss: 0.8407 - acc: 0.5494 - val_loss: 0.8596 - val_acc: 0.5571
Epoch 63/99
6534/6534 [==============================] - 0s 16us/step - loss: 0.8415 - acc: 0.5496 - val_loss: 0.8391 - val_acc: 0.5576
Epoch 64/99
6534/6534 [==============================] - 0s 14us/step - loss: 0.8416 - acc: 0.5497 - val_loss: 0.8612 - val_acc: 0.5571
Epoch 65/99
6534/6534 [==============================] - 0s 13us/step - loss: 0.8404 - acc: 0.5497 - val_loss: 0.8534 - val_acc: 0.5576
Epoch 66/99
6534/6534 [==============================] - 0s 15us/step - loss: 0.8410 - acc: 0.5497 - val_loss: 0.8451 - val_acc: 0.5576
Epoch 67/99
6534/6534 [==============================] - 0s 

In [89]:
# 56% accuracy after 99 epochs
nnf.evaluate(X_test, y_test)

2179/2179 [==============================] - 0s 22us/step


[0.8634638084135454, 0.5571363010828841]

* Which hyper-parameters are important?

For neutral network, the number of neurons, activation function and loss function matter.

* What hyper-parameter values work best? 

Through all results we got above, it's apparent that when we chose 'tanh' activation function and 'mean_squared_error' loss function, the accuracy is highest, 83%.

* How the the neural network compare to the supervised learners in part A?   

Comparing to the supervised learners in part A, the neutral network works better than naive bayes, appropriate to K-Nearest Neighbors and worse than decision tree and random forest.


In [103]:
mlp=MLPClassifier()

In [104]:
tunedParamMlp={'hidden_layer_sizes':[(100,),(55,55),(20,20,20)],'activation':['logistic','relu','tanh','identity'],'alpha':[0.000001,0.00001,0.0001,0.001,0.01],'solver':['lbfgs','sgd','adam'],'batch_size':[100,200,300],'verbose':[True,False]}

In [105]:
mlpTuned=RandomizedSearchCV(mlp,tunedParamMlp,cv=10)

In [106]:
mlpTuned.fit(X_train,y_train)

Iteration 1, loss = 1.88108151
Iteration 2, loss = 1.61648404
Iteration 3, loss = 1.57029400
Iteration 4, loss = 1.56001696
Iteration 5, loss = 1.55738079
Iteration 6, loss = 1.55567102
Iteration 7, loss = 1.55453287
Iteration 8, loss = 1.55454607
Iteration 9, loss = 1.55609696
Iteration 10, loss = 1.55532261
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 1, loss = 1.83293918
Iteration 2, loss = 1.58672112
Iteration 3, loss = 1.56041211
Iteration 4, loss = 1.55590446
Iteration 5, loss = 1.55487199
Iteration 6, loss = 1.55506084
Iteration 7, loss = 1.55462504
Iteration 8, loss = 1.55474503
Iteration 9, loss = 1.55460667
Iteration 10, loss = 1.55474802
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 1, loss = 1.91054767
Iteration 2, loss = 1.61434833
Iteration 3, loss = 1.56811052
Iteration 4, loss = 1.55936212
Iteration 5, loss = 1.55834594
Iteration 6, loss = 1.55623549
Iteration 

Iteration 1, loss = 27.84054827
Iteration 2, loss = 23.75725000
Iteration 3, loss = 16.08011090
Iteration 4, loss = 16.64986802
Iteration 5, loss = 14.30660072
Iteration 6, loss = 11.71511416
Iteration 7, loss = 17.77679826
Iteration 8, loss = 18.01223604
Iteration 9, loss = 13.90686724
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 1, loss = 21.72609168
Iteration 2, loss = 16.19873107
Iteration 3, loss = 12.54581422
Iteration 4, loss = 11.24075263
Iteration 5, loss = 12.09754713
Iteration 6, loss = 12.89211031
Iteration 7, loss = 10.80658866
Iteration 8, loss = 12.60807173
Iteration 9, loss = 14.14319145
Iteration 10, loss = 13.43342864
Training loss did not improve more than tol=0.000100 for two consecutive epochs. Stopping.
Iteration 1, loss = 27.41564672
Iteration 2, loss = 29.01419318
Iteration 3, loss = 28.28589850
Iteration 4, loss = 23.64766895
Iteration 5, loss = 13.14668812
Iteration 6, loss = 12.63668525
Iteration 7, loss

RandomizedSearchCV(cv=10, error_score='raise',
          estimator=MLPClassifier(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(100,), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='adam', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False),
          fit_params=None, iid=True, n_iter=10, n_jobs=1,
          param_distributions={'hidden_layer_sizes': [(100,), (55, 55), (20, 20, 20)], 'activation': ['logistic', 'relu', 'tanh', 'identity'], 'alpha': [1e-06, 1e-05, 0.0001, 0.001, 0.01], 'solver': ['lbfgs', 'sgd', 'adam'], 'batch_size': [100, 200, 300], 'verbose': [True, False]},
          pre_dispatch='2*n_jobs', random_state=None, refit=True,
          return_train_score='warn', scoring=None, verbose=0)

In [107]:
mlpTuned.best_params_

{'verbose': False,
 'solver': 'adam',
 'hidden_layer_sizes': (100,),
 'batch_size': 100,
 'alpha': 0.01,
 'activation': 'tanh'}

In [108]:
mlpTuned.best_score_

0.8258340985613712

In [110]:
mlpN=MLPClassifier(verbose=False,solver='adam',hidden_layer_sizes=(100,),batch_size=100,alpha=0.01,activation='tanh')

print("KfoldCrossVal mean score using Multi-layer Perceptron classifier is %s" %cross_val_score(mlpN,X,y,cv=10).mean())


sm= mlpN.fit(X_train, y_train)

y_pred = sm.predict(X_test)
print("Accuracy score using Multi-layer Perceptron classifier is %s" %metrics.accuracy_score(y_test, y_pred))

KfoldCrossVal mean score using Multi-layer Perceptron classifier is 0.8256565229948473
Accuracy score using Multi-layer Perceptron classifier is 0.817806333180358


* Which hyper-parameters are important?

For nMulti-layer Perceptron classifier, verbose, solver, hidden_layer_sizes, batch_size, alpha, activation matter.

* What hyper-parameter values work best? 

Through all results we got above, it's apparent that when 'verbose': False,'solver': 'adam', 'hidden_layer_sizes': (100,),'batch_size': 100,'alpha': 0.01,'activation': 'tanh', the accuracy is highest, 83%.

* How the the neural network compare to the supervised learners in part A?   

Comparing to the supervised learners in part A, the neutral network works better than naive bayes, appropriate to K-Nearest Neighbors and worse than decision tree and random forest.

## Stacked ensemble super-model 

##### mlxtend.StackingClassifier()

In [113]:
lr = LogisticRegression()
sclf = StackingClassifier(classifiers=[rdfN,mlpN], meta_classifier=lr)

In [115]:
print('10-fold cross validation:\n')
for clf, label in zip([rdfN,mlpN, sclf], 
                      ['Random Forest', 
                       'Neutral Network',
                       'StackingClassifier']):

    scores = model_selection.cross_val_score(clf, X, y, 
                                              cv=10, scoring='accuracy')
    print("Accuracy: %0.2f (+/- %0.2f) [%s]" 
          % (scores.mean(), scores.std(), label))

10-fold cross validation:

Accuracy: 0.99 (+/- 0.00) [Random Forest]
Accuracy: 0.83 (+/- 0.03) [Neutral Network]
Accuracy: 0.99 (+/- 0.00) [StackingClassifier]


Cause the accuracy of random forest equals to the accuracy of stacking classifier, I could not figure out whether stacking network helps.So here I chose knn to do a test.

In [116]:
sclfK = StackingClassifier(classifiers=[knn,mlpN], meta_classifier=lr)

In [117]:
print('10-fold cross validation:\n')
for clf, label in zip([knn,mlpN, sclf], 
                      ['K-Nearest Neighbors', 
                       'Neutral Network',
                       'StackingClassifier']):

    scores = model_selection.cross_val_score(clf, X, y, 
                                              cv=10, scoring='accuracy')
    print("Accuracy: %0.2f (+/- %0.2f) [%s]" 
          % (scores.mean(), scores.std(), label))

10-fold cross validation:

Accuracy: 0.83 (+/- 0.03) [K-Nearest Neighbors]
Accuracy: 0.83 (+/- 0.03) [Neutral Network]
Accuracy: 0.99 (+/- 0.00) [StackingClassifier]


* Did the stacked ensemble super-model help?

Through the test, it's obvious that the stacked ensemble super-model helps.

* How did to combine the models?  

Using mlxtend.StackingClassifier() to combine the best learners in part A & B.

* Cross-validate the model. How well did it do?   

Through 10 fold cross validation, it's apparent that the accuracy is 0.99 which means the model's accuracy is very high.